In [ ]:
PATH_DATA = "/home/a.makarchuk@rit.va/Desktop/kaggle-CV-best11/dogs_vs_cats/data/"
%cd $PATH_DATA

In [ ]:
import os

import pandas as pd


pd_ = pd.read_csv(PATH_DATA + "sampleSubmission.csv")
list_train_imgs = os.listdir(PATH_DATA + "train")
list_test_imgs = os.listdir(PATH_DATA + "test1")
print(sum(pd_["label"] == 0))

In [ ]:
import random

import cv2
import matplotlib.pyplot as plt


image_size = (128, 128)

rand_img_train_path = PATH_DATA + "train/" + random.choice(list_train_imgs)
img = plt.imread(rand_img_train_path)
img = cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)

print(rand_img_train_path)

plt.imshow(img)
plt.show()

In [ ]:
import numpy as np
import torch
import torch.nn as nn


def load_and_transform_img(path_img, image_size):
    img_name = path_img.split("/")[-1]
    label = 1 if img_name.split(".")[0] == "dog" else 0
    img = plt.imread(path_img)
    img = cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32) / 255.0
    return torch.from_numpy(img).float(), torch.from_numpy(np.array(label)).float()


def generate_batch(list_imgs_path, batch_size):
    curr_id = 0
    while curr_id + batch_size < len(list_imgs_path):
        images_tensor_list = list()
        labels_tensor_list = list()
        for i in range(batch_size):
            img_tensor, label = load_and_transform_img(
                PATH_DATA + "train/" + list_imgs_path[curr_id + i], image_size
            )
            images_tensor_list.append(img_tensor)
            labels_tensor_list.append(label)
        yield torch.stack(images_tensor_list), torch.stack(labels_tensor_list).unsqueeze(-1)
        curr_id += batch_size

In [ ]:
# check image loader with the generator


for i, (stacked_imgs, stacked_lbls) in enumerate(generate_batch(list_train_imgs, 1)):
    img = stacked_imgs.squeeze().numpy()
    label = stacked_lbls.squeeze().numpy()
    img = (img * 255).astype(int)
    print(label)
    plt.imshow(img)
    plt.show()
    if i > 10:
        break

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, image_size=(128, 128)):
        super().__init__()
        h, w = image_size
        self.flatten = nn.Flatten()
        self.base_classifier = nn.Sequential(
            nn.BatchNorm1d(h * w * 3),
            nn.Linear(h * w * 3, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        out_logits = self.base_classifier(x)
        return out_logits

In [ ]:
from sklearn.metrics import f1_score
from tqdm import tqdm


epochs = 10
batch_size = 100
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork().to(device)
sigmoid = nn.Sigmoid()
opt = torch.optim.Adam(model.parameters(), lr=5e-3)
loss = nn.BCELoss()

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
np.random.seed(seed)
random.seed(seed)

train_ratio = 0.85
list_imgs = list_train_imgs[:]
random.shuffle(list_imgs)
list_train_imgs = list_imgs[: int(len(list_imgs) * train_ratio)]
list_val_imgs = list_imgs[int(len(list_imgs) * train_ratio) : len(list_imgs)]

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")

    model.train()
    for idx, (batch_imgs, lbl) in tqdm(
        enumerate(generate_batch(list_train_imgs, batch_size)),
        total=len(list_train_imgs) // batch_size,
    ):
        batch_imgs = batch_imgs.to(device)
        lbl = lbl.to(device)
        logits = model(batch_imgs)

        output = loss(sigmoid(logits), lbl)
        output.backward()
        opt.step()
        opt.zero_grad()

        predicted_probs = sigmoid(logits).detach().cpu().numpy()
        predicted_labels = (predicted_probs > 0.5).astype(float)

        if idx % 50 == 0:
            print(
                f"Train F1-score: {f1_score(lbl.detach().cpu().numpy(), predicted_labels):.4f} Train Loss: {output:.4f}"
            )

    model.eval()
    val_preds = []
    val_labels = []
    val_loss = []
    with torch.no_grad():
        for batch_imgs, lbl in tqdm(
            generate_batch(list_val_imgs, batch_size), total=len(list_val_imgs) // batch_size
        ):
            batch_imgs = batch_imgs.to(device)
            lbl = lbl.to(device)

            logits = model(batch_imgs)
            val_loss.append(loss(sigmoid(logits), lbl).detach().cpu().numpy())
            predicted_probs = sigmoid(logits).detach().cpu().numpy()
            predicted_labels = (predicted_probs > 0.5).astype(float)

            val_preds.extend(predicted_labels)
            val_labels.extend(lbl.detach().cpu().numpy())

    val_f1 = f1_score(val_labels, val_preds)
    print(f"Validation F1-score: {val_f1:.4f} Validation Loss: {np.mean(val_loss):.4f}")

In [ ]:
model.eval()
test_preds = []
sigmoid = nn.Sigmoid()

pd_test = pd.DataFrame(columns=["id", "label"])

with torch.no_grad():
    for path_img in tqdm(list_test_imgs, total=len(list_test_imgs)):
        id_img = path_img.split(".")[0]
        img = plt.imread(PATH_DATA + "test1/" + path_img)
        img = cv2.resize(img, image_size, interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32) / 255.0
        logits = model(torch.from_numpy(img).unsqueeze(0))
        pd_test.loc[len(pd_test)] = [id_img, sigmoid(logits).item()]

In [ ]:
pd_test

In [ ]:
pd_test.to_csv("base_from_scratch.csv", index=False)